In [70]:
from datetime import datetime
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rio
import os
from pathlib import Path
import sys
from datetime import timedelta

path_mod = f"{Path(os.path.dirname(os.path.abspath(''))).parents[2]}/"
sys.path.append(path_mod)

from src.indicators.flooding.config import Config
from src.indicators.flooding.floodscan import floodscan
from src.indicators.flooding.glofas import utils

config = Config()
mpl.rcParams['figure.dpi'] = 300

PLOT_DIR = config.DATA_DIR / 'processed' / 'mwi' / 'plots' / 'flooding'
PRIVATE_DIR = config.DATA_PRIVATE_DIR
EXPLORE_DIR = PRIVATE_DIR / 'exploration' / 'mwi' / 'flooding'

In [45]:
mwi_shp = gpd.read_file(
    config.DATA_DIR / 
    'raw' / 
    'mwi' / 
    'cod_ab' / 
    'mwi_adm_nso_20181016_shp' / 
    'mwi_admbnda_adm2_nso_20181016.shp'
)
mwi_shp_sel = mwi_shp.loc[mwi_shp['ADM2_EN'].isin(['Chikwawa', 'Nsanje'])]

Create dataset of Glofas points that we're using.

In [32]:
# Coordinates pulled from GloFAS site _lis is for the LISFLOOD coords
glofas_points = pd.DataFrame(
    {'Station_ID': ['G2001', 'G1724'],
     'Lat': [-16.25, -16.52],
     'Lon': [34.97, 35.15],
     'Lat_lis' : [-16.25, -16.45],
     'Lon_lis': [34.95, 35.05]
    })

glofas_points = gpd.GeoDataFrame(
    glofas_points, geometry=gpd.points_from_xy(glofas_points.Lon_lis, glofas_points.Lat_lis))

For plotting Floodscan data, let's get the dates where GloFAS exceeds a 1 in 2 year return period threshold. We'll focus specifcally in the northernmost GloFAS station, G2001. 

In [138]:
rp = 2
duration = 1
station = 'G2001'

ds_glofas_reanalysis = utils.get_glofas_reanalysis(
    country_iso3='mwi')
df_return_period = utils.get_return_periods(ds_glofas_reanalysis, method='analytical')
rp_val = df_return_period.loc[rp, station]

glofas_events = utils.get_dates_list_from_data_array(
            da=ds_glofas_reanalysis[station], 
            threshold=rp_val,
            min_duration=duration
        )
glofas_events = np.array(glofas_events)[glofas_events > fs_raw.time.min().values]

Clean up the Floodscan data

In [134]:
fs = floodscan.Floodscan()
fs_raw = fs.read_raw_dataset()

# Manually define a clipping bounding box for the raster data
clip_geometry = [
    {
        'type': 'Polygon',
        'coordinates': [
            [[34.1011082082,-17.12960312],
             [35.4722024947,-17.12960312],
             [35.4722024947,-15.7014191359],
             [34.1011082082,-15.7014191359],
             [34.1011082082,-17.12960312]
            ]]
    }
]

In [139]:
for event in glofas_events: 
    t_start = event
    t_end = event + np.timedelta64(9, 'D')

    ds_sel = (
        fs_raw
        .sel(time=slice(t_start, t_end))[['SFED_AREA']]
        .to_array()
        .rio.write_crs("EPSG:4326")
        .rio.set_spatial_dims(x_dim="lon", y_dim="lat")
        .rio.clip(clip_geometry)
    )

    p = ds_sel.plot(col="time", col_wrap=5, vmin=0, vmax=1)
    
    for ax in p.axes.flat:
        mwi_shp_sel.boundary.plot(ax=ax)
        glofas_points.plot(ax=ax, color='red')

    t = pd.to_datetime(str(event))
    date = t.strftime('%Y-%m-%d')
    plt.savefig(PLOT_DIR / f'floodscan__{station}_{date}.png')
    plt.close()

Looking through these plots, we want to try and find Floodscan-derived indicators that would logically be indicative of significant flooding and would be correlated with Glofas streamflow peaks at the return period that we've specified. Some potentially valuable Floodscan indicators to explore might be...

- % of cells w flooded fraction > 20% --> aggregated across both Chikwawa and Nsanje

ALSO, what if we're thinking about streamflow wrong... what if it isn't necessarily a peak in a single day that causes flooding, but a more sustained and less extreme increase in streamflow over x days?